[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/polislizarralde/PythonPlayground/blob/main/docs/PlagueProject/FittingMetapopModelPlague.ipynb)

In [1]:
%matplotlib inline


In [2]:
# Python 3.11.2
# Import packages
import scipy.integrate as scipy
import scipy.optimize as optimize
import scipy.stats as stats
import pandas as pd
import numpy as np
import pylab as pl
import random
import matplotlib.pyplot as plt
from collections import defaultdict
import json  # for pretty printing

import shutil
import sys
import os.path


Initializing the population size and initial conditions:

In [3]:
# vector of population sizes with lenght n
n = 19
N = np.array([204  # Bromma
              , 217  # Oja
              , 1895  # S Maria Ystad 1749
              , 554  # Valleberga
              , 693  # S Kopinge
              , 403  # Horups
              , 582  # Bjäresjö 1780
              , 716  # Villie 1749
              , 418  # Snårestad 1775
              , 519  # Skårby 1749
              , 262  # Hammenhögs 1749
              , 560  # Glemminge 1775
              , 236  # Balkåkra 1775
              , 334  # Baldringe 1749
              , 299  # Ovraby
              , 761  # Sövestads 1749
              , 776  # Löderups 1749
              , 951  # Borrby 1775
              , 358  # Tosterups 1775
              ])
# Initial conditions for each patch

I0 = np.zeros(n)  # vector of initial infecteds with lenght n
I0[0] = 1.0  # the first element of the I0 vector is set to 1

S0 = np.zeros(n)  # vector of initial susceptibles with lenght n
for i in range(n):
    S0[i] = N[i] - I0[i]

R0 = np.zeros(n)  # vector of initial removeds with lenght n
D0 = np.zeros(n)  # vector of initial deaths with lenght n

# print(S0,I0,R0,D0)


In [4]:
# Defining the transmission rate matrix as a function of two parameters

def TransmissionRateMatrix(beta: float, p: float) -> np.ndarray:
    return (
        np.array([
            [beta, p, 0, 0, 0, 0, p, 0, 0, 0, 0, 0, 0, 0, 0, p, 0, 0, 0],
            [p, beta, p, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, p, beta, 0, p, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, beta, 0, p, 0, 0, 0, 0, 0, p, 0, 0, 0, 0, p, 0, 0],
            [0, 0, p, 0, beta, 0, 0, 0, 0, 0, 0, p, 0, 0, p, 0, 0, 0, p],
            [0, 0, 0, p, 0, beta, 0, 0, 0, 0, p, p, 0, 0, 0, 0, p, p, 0],
            [p, 0, 0, 0, 0, 0, beta, 0, 0, p, 0, 0, p, 0, 0, p, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, beta, 0, p, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, beta, 0, 0, 0, p, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, p, p, 0, beta, 0, 0, p, 0, 0, p, 0, 0, 0],
            [0, 0, 0, 0, 0, p, 0, 0, 0, 0, beta, 0, 0, 0, 0, 0, 0, p, 0],
            [0, 0, 0, p, p, p, 0, 0, 0, 0, 0, beta, 0, 0, 0, 0, 0, 0, p],
            [0, 0, 0, 0, 0, 0, p, 0, p, p, 0, 0, beta, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, beta, 0, p, 0, 0, 0],
            [0, 0, 0, 0, p, 0, 0, 0, 0, 0, 0, 0, 0, 0, beta, 0, 0, 0, p],
            [p, 0, 0, 0, 0, 0, p, 0, 0, p, 0, 0, 0, p, 0, beta, 0, 0, 0],
            [0, 0, 0, p, 0, p, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, beta, p, 0],
            [0, 0, 0, 0, 0, p, 0, 0, 0, 0, p, 0, 0, 0, 0, 0, p, beta, 0],
            [0, 0, 0, 0, p, 0, 0, 0, 0, 0, 0, p, 0, 0, p, 0, 0, 0, beta]
        ])
    )


Define the data. In this case we have:
    1. Total number of deaths per patch
    2. Initial day of the outbreak for each patch
    3. Last day of the outbreak for each patch

'0' indicates missing information

In [5]:
# Deaths by parish. Zero means we don't have information
TotalDeathsByParish = 1.0 * \
    np.array([0, 40, 740, 0, 80, 60, 0, 0, 38,
             70, 0, 0, 45, 0, 0, 0, 270, 45, 0])
# initial day for each parish
BeginPlagueByParish: list[int] = [1, 61, 61, 91, 91, 91, 91,
                                  121, 121, 121, 121, 121, 121, 121, 151, 181, 271, 331, 511]
# final day for each parish. Zero means we don't have information
EndPlagueByParish: list[int] = [60  # Bromma
                                , 360  # Oja
                                , 270  # Ystad
                                , 0  # Valleberga
                                , 300  # Kopinge
                                , 0  # Horups
                                , 0  # Bjaresjo
                                , 150  # Villie
                                , 510  # Snarestad
                                , 270  # Skarby
                                , 0  # Hammenhogs
                                , 150  # Glemminge
                                , 240  # Balkakra
                                , 0  # Baldringe
                                , 180  # Ovraby
                                , 270  # Sovestads
                                , 570  # Loderups
                                , 570  # Borrby
                                , 570  # Tosterups
                                ]


Generating the differential equations

In [6]:
def SIRD_model(y, t, model_parameters: tuple[dict]):

    parameters = model_parameters[0]
    beta: float = parameters['beta']
    gamma: float = parameters['gamma']
    p: float = parameters['p']
    mu: float = parameters['mu']
    N = parameters['N']
    n: int = parameters['n']

    S = defaultdict(float)
    I = defaultdict(float)
    R = defaultdict(float)
    D = defaultdict(float)

    vars = tuple(sum([[S[i], I[i], R[i], D[i]] for i in range(n)], []))
    vars = y

   # Choosing the corresponding output for each subpopulation
    def entryS(i):
        return vars[4 * i]

    def entryI(i):
        return vars[4 * i + 1]

    def entryR(i):
        return vars[4 * i + 2]

    def entryD(i):
        return vars[4 * i + 3]

    # Initializando the directory for each subpopulation
    dS = {}
    dI = {}
    dR = {}
    dD = {}

    # Defining the differential equations for each subpopulation
    for i in range(n):
        dS[i] = - entryS(i) / (N[i]*1.0) * sum(TransmissionRateMatrix(beta, p)
                                               [i][j] * entryI(j) for j in range(n))
        dI[i] = entryS(i) / (N[i]*1.0) * sum(TransmissionRateMatrix(beta, p)
                                             [i][j] * entryI(j) for j in range(n)) - gamma * entryI(i)
        dR[i] = gamma * mu * entryI(i)
        dD[i] = gamma * (1 - mu) * entryI(i)

    derivates = sum([[dS[i], dI[i], dR[i], dD[i]] for i in range(n)], [])
    return derivates   # For odeint


def generate_sol(genInput: dict) -> dict:
    init = genInput['init']
    n = genInput['n']

    # Initial conditions vector for the metapopulation model. len(initConditions) = 4*n
    initConditions = tuple(sum(
        [[init['S'][i], init['I'][i], init['R'][i], init['D'][i]] for i in range(n)], []))

    # Time vector
    T = genInput['T']
    t = np.linspace(0, T, T+1)

    # Computing the numerical solution
    model = genInput['model']
    solution = scipy.odeint(model, initConditions, t, args=((genInput,),))

    indexVar = {}
    indexVar['S'] = 0
    indexVar['I'] = 1
    indexVar['R'] = 2
    indexVar['D'] = 3

    def varSol(patch, var):
        return solution[:, 4*patch + indexVar[var]]

    return {'S': {patch: varSol(patch, 'S') for patch in range(n)},
            'I': {patch: varSol(patch, 'I') for patch in range(n)},
            'R': {patch: varSol(patch, 'R') for patch in range(n)},
            'D': {patch: varSol(patch, 'D') for patch in range(n)},
            'N': genInput['N'],
            'init': init,
            'beta': genInput['beta'],
            'gamma': genInput['gamma'],
            'mu':  genInput['mu'],
            't': t,
            'n': n,
            'p': genInput['p'],
            'model': model,
            'raw_solution': solution}


In [6]:

# Define the objective function to minimize (sum of squared errors)
def objectiveFunction(parameters, args):
    beta, p, gamma, mu = parameters
    beginTime, endTime, deathData = args
    model_info = {'model': SIRD_model,
                  'init': {
                      'S': S0,
                      'I': I0,
                      'R': R0,
                      'D': D0,
                  },  # defining the initial values for the model
                  'beta': beta,
                  'p': p,
                  'gamma': gamma,
                  'mu': mu,
                  'N': N,
                  'n': 19,`z


In [7]:
# Set up the data to fit
beginTime = BeginPlagueByParish
endTime = EndPlagueByParish
deathData = TotalDeathsByParish


# Choose initial guesses for the parameters to fit
beta_guess = 0.5
p_guess = 0.1
gamma_guess = 0.5
mu_guess = 0.5

# Minimize the objective function to obtain estimates for beta and gamma

result = optimize.minimize(
    lambda x: objectiveFunction(x, (beginTime, endTime, deathData)),
    (beta_guess, p_guess, gamma_guess, mu_guess),
    method='L-BFGS-B',
    bounds=[(0, 2), (0, 1), (0, 1), (0, 1)])
beta_estimated, p_estimated, gamma_estimated, mu_estimated = result.x

print("beta = ", beta_estimated)
print("p = ", p_estimated)
print("gamma = ", gamma_estimated)
print("mu = ", mu_estimated)


beta =  0.594647017837068
p =  1.7008299978816306e-09
gamma =  0.42123719151572153
mu =  0.4922276846825045


In [7]:
# Results from the optimization
beta_estimated = 0.5962793770426265
p_estimated = 1.7581613689424408e-06
gamma_estimated = 0.4214334954023917
mu_estimated = 0.4867362091247948


In [8]:
# Set up the data to fit
beginTime = BeginPlagueByParish
endTime = EndPlagueByParish
deathData = TotalDeathsByParish

# Choose initial guesses for the parameters to fit
beta_guess = 0.5
p_guess = 0.1
gamma_guess = 0.5
mu_guess = 0.5

objectiveFunction((beta_guess, p_guess, gamma_guess, mu_guess),
                  (beginTime, endTime, deathData))


NameError: name 'objectiveFunction' is not defined

In [10]:
objectiveFunction((beta_estimated, p_estimated, gamma_estimated,
                  mu_estimated), (beginTime, endTime, deathData))


101710.95485988606

In [9]:
model_estimation = {'model': SIRD_model,
                    'init': {
                        'S': S0,
                        'I': I0,
                        'R': R0,
                        'D': D0,
                    },  # defining the initial values for the model
                    'beta': beta_estimated,
                    'p': p_estimated,
                    'gamma': gamma_estimated,
                    'mu': mu_estimated,
                    'N': N,
                    'n': 19,
                    'T': 700}
model_dict = generate_sol(model_estimation)


Computing the daily deaths from the model output for a given patch

In [10]:
def daily_deaths(solution_model: dict, patch: int, T_inf: int, T_sup: int) -> list[float]:
    assert T_inf >= 0, "t must be greater than 0"
    assert T_sup >= 0, "t must be greater than 0"
    cumulative_deaths = solution_model['D'][patch]  # list of floats
    return [cumulative_deaths[t+1] - cumulative_deaths[t]
            for t in range(T_inf, T_sup)]


Plotting the solution

In [16]:
patchNames = ["Bromma", "Oja", "Ystad", "Valleberga", "S. Kopinge", "Horups", "Bjäresjö", "Villie", "Snårestad",
              "Skårby", "Hammenhögs", "Glemminge", "Balkåkra", "Baldringe", "Ovraby", "Sövestads", "Löderups", "Borrby", "Tosterups"]


def plot_SIRD_solution(model: dict, state: list[str] = ['S', 'I', 'R', 'D', 'DailyDeaths']):

    for key in state:
        if key not in model:
            raise ValueError(f"Invalid state: {key}")
        for i in range(model['n']):
            plt.plot(model['t'], model[key][i],
                     label=f'{key} - {patchNames[i]}')
    plt.xlabel('Time')
    plt.ylabel('Infectious')
    plt.title('SIRD model')
    plt.legend()
    plt.show()


In [ ]:
# Number of patches
n = model_estimation['n']

# Set the figsize for each subplot
figsize_single_subplot = (8, 2)

# Calculate the total figure height based on the number of subplots and their height
fig_height = figsize_single_subplot[1] * n


# Create a figure and an array of axes with nrows=n and ncols=1
fig, axes = plt.subplots(nrows=n, ncols=1, figsize=(
    figsize_single_subplot[0], fig_height), sharex=False)

tick_positions = [30, 60, 90, 120, 150, 180, 210, 240,
                  270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570]
tick_labels = ["Apr 1712", "May 1712", "Jun 1712", "Jul 1712", "Aug 1712", "Sep 1712", "Oct 1712", "Nov 1712", "Dec 1712", "Jan 1713", "Feb 1713", "Mar 1713", "Apr 1713", "May 1713", "Jun 1713", "Jul 1713", "Aug 1713", "Sep 1713", "Oct 1713"
               ]

# Plot daily deaths for each patch i
for i in range(n):
    if TotalDeathsByParish[i] != 0 and EndPlagueByParish[i] != 0:
        initial_position = BeginPlagueByParish[i]
        final_position = EndPlagueByParish[i]
        # axes[i].plot(initial_position,1, 'bo', label='Observed data')
        axes[i].plot(final_position, TotalDeathsByParish[i],
                     'bo', label='Observed data')
        axes[i].plot(model_dict['D'][i], color='orange', label=(patchNames[i]))
        # axes[i].set_xlabel('Day')
        axes[i].set_ylabel('Cumulative Deaths', font='Helvetica')
        axes[i].legend(loc='lower right')
        axes[i].xaxis.set_ticks(
            tick_positions, tick_labels, rotation=70, font='Helvetica', fontsize=9)
    else:
        axes[i].plot(model_dict['D'][i], color='orange', label=(patchNames[i]))
        # axes[i].set_xlabel('Day')
        axes[i].set_ylabel('Cumulative Deaths', font='Helvetica')
        axes[i].legend(loc='lower right')
        axes[i].xaxis.set_ticks(
            tick_positions, tick_labels, rotation=70, font='Helvetica', fontsize=9)

# Adjust the layout to avoid overlapping
plt.tight_layout()
plt.show()


In [ ]:
# Set the figsize for each subplot
figsize_single_subplot = (8, 2)

# Calculate the total figure height based on the number of subplots and their height
fig_height = figsize_single_subplot[1] * n

# Create a figure and an array of axes with nrows=n and ncols=1
fig, axes = plt.subplots(nrows=n, ncols=1, figsize=(
    figsize_single_subplot[0], fig_height))

tick_positions = [30, 60, 90, 120, 150, 180, 210, 240,
                  270, 300, 330, 360, 390, 420, 450, 480, 510, 540, 570]
tick_labels = ["Apr 1712", "May 1712", "Jun 1712", "Jul 1712", "Aug 1712", "Sep 1712", "Oct 1712", "Nov 1712", "Dec 1712", "Jan 1713", "Feb 1713", "Mar 1713", "Apr 1713", "May 1713", "Jun 1713", "Jul 1713", "Aug 1713", "Sep 1713", "Oct 1713"
               ]

for i in range(model_dict['n']):
    # Plot daily deaths for each patch i
    axes[i].plot(daily_deaths(model_dict, i, 0, 570),
                 color='blue', label=(patchNames[i]))
    axes[i].set_ylabel('Daily Deaths', font='Helvetica')
    axes[i].legend(loc='upper right')
    axes[i].xaxis.set_ticks(tick_positions, tick_labels,
                            rotation=70, font='Helvetica', fontsize=9)

# Adjust the layout to avoid overlapping
plt.tight_layout()
plt.show()
